## Docker Deployment for
#### Pipeline segment: **Data Preprocessing**
---

In [ ]:
%%bash

image=customer-churn-prediction-preprocessing-image

cd customer_churn_inference_preprocessing/

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
#region=${region:-us-west-1}


fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${image}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${image}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region "${region}" | docker login --username AWS --password-stdin "${account}".dkr.ecr."${region}".amazonaws.com

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${image} .
docker tag ${image} ${fullname}

docker push ${fullname}

## Docker Deployment for
#### Pipeline segment: **Training**
---

In [2]:
%%bash

image=customer-churn-prediction-training-image

cd awsworkshop_team2_tr_training/

chmod +x model/

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi


# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-ap-southeast-1}


fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${image}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${image}" > /dev/null
fi

# Get the login command from ECR and execute it directly
#aws ecr get-login-password --region "${region}" | docker login --username AWS --password-stdin "${account}".dkr.ecr."${region}".amazonaws.com

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
aws ecr get-login-password --region "${region}" | docker login --username AWS --password-stdin "${account}".dkr.ecr."${region}".amazonaws.com

docker build  -t ${image} ./
docker tag ${image} ${fullname}

docker push ${fullname}


Login Succeeded
Sending build context to Docker daemon  26.11kB
Step 1/12 : FROM ubuntu:20.04
 ---> ff0fea8310f3
Step 2/12 : ENV LANG=C.UTF-8 LC_ALL=C.UTF-8
 ---> Using cache
 ---> 876c55fa3933
Step 3/12 : RUN apt-get update --fix-missing && apt-get install -y wget bzip2 ca-certificates     libglib2.0-0 libxext6 libsm6 libxrender1
 ---> Using cache
 ---> 9079c3fa1228
Step 4/12 : RUN apt-get update && apt-get install -y --no-install-recommends      python3      python3-pip      python3-dev      &&      apt-get clean &&      rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 29daeefc8793
Step 5/12 : RUN apt-get update --fix-missing &&     apt-get -y install gcc g++ make
 ---> Using cache
 ---> 48b08c60339c
Step 6/12 : ARG PIP=pip3
 ---> Using cache
 ---> b0dc47259fc8
Step 7/12 : WORKDIR /
 ---> Using cache
 ---> 6da86a3b1de8
Step 8/12 : RUN ${PIP} install --no-cache --upgrade             lightgbm             tqdm             xgboost             seaborn             sklearn             py

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

